### Portfolio Theory File Builder

Modern portfolio theory (MPT) is a theory on how risk-averse investors can construct portfolios to maximize expected return based on a given level of market risk.

In this file, I create functions to calculate the expected risk and return of a stock. Expected returns is calculated using the Capital Asset Pricing Model (CAPM) and the expected risk is calculated using the variance of a stock and the covariance of two stocks.

Stock market data will focus on the last five years, with the exception of the Market Returns and the Risk Free Rate.

In [1]:
# Imported Libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os

In [2]:
def historicalMarketRate(stock):
    year_starts = ['2011-01-03','2012-01-03','2013-01-02','2014-01-02','2015-01-02',
        '2016-01-04','2017-01-03','2018-01-02','2019-01-02','2020-01-02','2021-01-04']
   
    historical = stock.history(start = '2011-01-01', end = '2021-01-10')
    
    sum = 0
    count = 0
    for i in range(10):
        year_return = (historical.loc[year_starts[i+1]]['Close'] - historical.loc[year_starts[i]]['Close']) / historical.loc[year_starts[i]]['Close']
        sum = sum + year_return
        count = count + 1
    
    return sum / count

def covariance(stock1, stock2):
    stock1_close = stock1.history(period = '5y')['Close']
    stock1_close_mean = stock1_close.mean()

    stock2_close = stock2.history(period = '5y')['Close']
    stock2_close_mean = stock2_close.mean()
    sum = 0
    count = 0

    for i in range(len(stock1_close)):
        sum = sum + (stock1_close[i] - stock1_close_mean)*(stock2_close[i] - stock2_close_mean)
        count = count + 1

    return sum / (count - 1)

def variance(stock):
    stock_close = stock.history(period = '5y')['Close']
    stock_close_mean = stock_close.mean()
    sum = 0
    count = 0

    for entry in stock_close:
        sum = sum + (entry-stock_close_mean)**2
        count = count + 1

    return sum / (count - 1)

def beta(stock):

    sp500 = yf.Ticker("^GSPC")
    return covariance(stock, sp500) / variance(sp500)

def riskfreerate(stock):
    return stock.history(period = '10y')['Close'].mean()/100

def expectedReturn(stock):
    tbill = yf.Ticker("^TNX")
    sp500 = yf.Ticker("^GSPC")
    rf = riskfreerate(tbill)
    rm = historicalMarketRate(sp500)
    stock_beta = beta(stock)
    return rf + stock_beta*(rm - rf)

def riskReturnPortfolio(ticker1, ticker2):
    
    stock1 = yf.Ticker(ticker1)
    stock2 = yf.Ticker(ticker2)
    
    stock1_ER = expectedReturn(stock1)
    stock2_ER = expectedReturn(stock2)
    stock1_var = variance(stock1)
    stock2_var = variance(stock2)
    stock_cov = covariance(stock1, stock2)

    stock1_weight = []
    stock2_weight = []
    portfolio_return = []
    portfolio_risk = []

    for i in np.arange(0,1.01,0.01):
        stock1_weight.append(i)
        stock2_weight.append(1-i)
        er = round(stock1_ER*i + stock2_ER*(1-i), 5)
        risk = round((i**2) * stock1_var + ((1-i)**2) * stock2_var + 2 * i * (1-i) * stock_cov, 5)
        portfolio_return.append(er)
        portfolio_risk.append(risk)

    data_dict = {'stock1_weight':stock1_weight, 'stock2_weight':stock2_weight, 
        'portfolio_return':portfolio_return, 'portfolio_risk':portfolio_risk}

    return pd.DataFrame(data_dict)   

def correlation(ticker1, ticker2):
    stock1 = yf.Ticker(ticker1)
    stock2 = yf.Ticker(ticker2)
    return round((covariance(stock1, stock2) / math.sqrt(variance(stock1) * variance(stock2))), 5)

def correlation_matrix(ticker_list):

    corr_matrix = np.empty((len(ticker_list), len(ticker_list)))

    for i in range(len(ticker_list)):
        for j in range(len(ticker_list)):
            corr_matrix[i,j] = correlation(ticker_list[i], ticker_list[j])

    df = pd.DataFrame(corr_matrix)
    df.columns = ticker_list
    df.index = ticker_list
    return df

def all_riskReturnPortfolio(ticker_list):
    df = pd.DataFrame(columns = ['stock_1', 'stock_2', 'stock_1_weight','stock_2_weight', 'portfolio_return', 'portfolio_risk'])

    for i in range(len(ticker_list)):
        for j in range(i+1, len(ticker_list)):
            temp_df = riskReturnPortfolio(ticker_list[i], ticker_list[j])

            for  pos, row in temp_df.iterrows():
                row_df = pd.DataFrame([[ticker_list[i], ticker_list[j], row.stock1_weight, row.stock2_weight, row.portfolio_return, row.portfolio_risk]], 
                        columns = ['stock_1', 'stock_2', 'stock_1_weight','stock_2_weight', 'portfolio_return', 'portfolio_risk'])
                df = df.append(row_df)

    return df



In [3]:
# Collecting List of Stocks from .csv files

tickers = [i[:-4] for i in os.listdir('data/')]
tickers.remove('sp500')
tickers

['amzn', 'bp', 'dis', 'ge', 'gs', 'pfe', 'pg', 'rio', 'ugi']

In [4]:
# Write Correlation Matrix to file
stockCorr = correlation_matrix(tickers)
stockCorr.to_csv('correlation.csv')

In [5]:
# Write Risk Return data to file
riskReturns = all_riskReturnPortfolio(tickers)
riskReturns.to_csv('stockRiskReturns.csv')